In [3]:
import keras
from tensorflow.keras import layers, activations
import tensorflow as tf
import matplotlib as plt
from tensorflow.keras.models import load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
import matplotlib.pyplot as plt 
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
#Constants
IMG_SIZE = 256
BATCH_SIZE = 32
NUM_CLASSES = 3
CLASSES = ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [12]:
# Train = keras.utils.image_dataset_from_directory(
#     directory = "/Coding Projects/DL/Potato Disease Classification/train",
# labels = "inferred",
# label_mode = "int",
# batch_size = 32,
# image_size = (256,256),
# shuffle = True,
# )
# num_classes = len(Train.class_names)
# print(num_classes)

# Validation = keras.utils.image_dataset_from_directory(
#     directory = "/Coding Projects/DL/Potato Disease Classification/validate",
# labels = "inferred",
# label_mode = "int",
# batch_size = 32,
# image_size = (256,256),
# shuffle = True,
# )

# Test = keras.utils.image_dataset_from_directory(
#     directory = "/Coding Projects/DL/Potato Disease Classification/test",
# labels = "inferred",
# label_mode = "int",
# batch_size = 32,
# image_size = (256,256),
# shuffle = True,
# )

Found 2124 files belonging to 3 classes.
3
Found 876 files belonging to 3 classes.
Found 276 files belonging to 3 classes.


In [13]:
Dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory = "Dataset",
    shuffle = True,
    batch_size = BATCH_SIZE,
    image_size = (256,256),
)

Found 2151 files belonging to 3 classes.


In [14]:
CLASS_NAMES = Dataset.class_names
CLASS_NAMES

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [26]:
resize_rescale = tf.keras.Sequential([
    layers.Resizing(IMG_SIZE, IMG_SIZE),
    layers.Rescaling(1.0/255)
])

In [27]:
dataAug = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
])

In [21]:
# # ##
# datagen = ImageDataGenerator(
#     rotation_range=30,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# input_directory = f'/Coding Projects/DL/Potato Disease Classification/Dataset/Potato___healthy'
# output_directory = f'/Coding Projects/DL/Potato Disease Classification/Dataset/Potato___healthy'
# os.makedirs(output_directory, exist_ok=True)


# for filename in os.listdir(input_directory):
 
        
#         img_path = os.path.join(input_directory, filename)
#         img = load_img(img_path)  # Load image
#         x = img_to_array(img) 
#         x = np.expand_dims(x, axis=0)  # Reshape
#         # Create an iterator that will produce augmented images
#         i = 0
#         for batch in datagen.flow(x, batch_size=1, save_to_dir=output_directory, save_prefix='aug', save_format='jpeg'):
#             i += 1
            
                
#             if i >= 3:  # Number of augmented images per input image
#                 break  # Exit the loop
# # ##

In [5]:
# def process(img, label):
#     image = tf.cast(img / 256., tf.float32)
#     return image, label

# Dataset = Dataset.map(process)


In [28]:
train_percent = 0.8
test_percent = 0.1
val_percent = 0.1
Dataset = Dataset.shuffle(10000, seed = 12)
train_size = int(len(Dataset)*train_percent)
test_size = int(len(Dataset)*test_percent)
val_size = int(len(Dataset)*val_percent) 
Train = Dataset.take(train_size)
rest = Dataset.skip(train_size)
rest = rest.shuffle(10000, seed = 12)
Test = rest.take(test_size)
rest = rest.skip(test_size)
Val = rest.take(test_size)
print(len(Train), len(Val), len(Test))

54 6 6


In [29]:
Train = Train.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
Test = Test.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
Val = Val.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)

In [33]:
model = keras.Sequential([resize_rescale, dataAug])

model.add(Conv2D(32, kernel_size=(3, 3),activation = "relu", input_shape=(256, 256)))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, kernel_size=(3, 3),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, kernel_size=(3, 3),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, kernel_size=(3, 3),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, kernel_size=(3, 3),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, kernel_size=(3, 3),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dense(64, activation="relu"))

model.add(Dense(NUM_CLASSES, activation="softmax"))

model.compile(optimizer="adam", loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

In [34]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_4 (Sequential)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_5 (Sequential)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_24 (MaxPooling2D)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_25 (MaxPooling2D)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_26 (MaxPooling2D)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_27 (Conv2D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_27 (MaxPooling2D)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_28 (MaxPooling2D)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_29 (MaxPooling2D)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [35]:
modelHistory = model.fit(Train, epochs=50, validation_data=Val, batch_size=BATCH_SIZE, verbose=1)

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 27s 418ms/step - accuracy: 0.4839 - loss: 0.9050 - val_accuracy: 0.6615 - val_loss: 0.6929
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 20s 378ms/step - accuracy: 0.7410 - loss: 0.5791 - val_accuracy: 0.7396 - val_loss: 0.4900
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 20s 375ms/step - accuracy: 0.8163 - loss: 0.4614 - val_accuracy: 0.8229 - val_loss: 0.4364
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 23s 432ms/step - accuracy: 0.8658 - loss: 0.3549 - val_accuracy: 0.8021 - val_loss: 0.5591
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 24s 445ms/step - accuracy: 0.9053 - loss: 0.2426 - val_accuracy: 0.9531 - val_loss: 0.1174
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 25s 460ms/step - accuracy: 0.9278 - loss: 0.2012 - val_accuracy: 0.9323 - val_loss: 0.2445
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 25s 468ms/step - accuracy: 0.9497 - loss: 0.1609 - val_accuracy: 0.9635 - val_loss: 0.1066
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 472ms/step - accuracy: 0.9558 - loss: 0.1265 - val_accu

In [36]:
model.evaluate(Test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.9724 - loss: 0.0776


[0.05970561131834984, 0.9791666865348816]

In [37]:
model.save("/Coding Projects/DL/Potato Disease Classification/models/1.keras")


In [27]:
from tensorflow.keras.preprocessing import image
import numpy as np
image_path = "/Coding Projects/DL/Potato Disease Classification/late.png"
img = image.load_img(image_path, target_size=(255, 255))

In [28]:
model = load_model('/Coding Projects/DL/Potato Disease Classification/models/1.keras')

In [29]:
img_array = image.img_to_array(img)

# Add an extra dimension to the array (since the model expects a batch)
img_array = np.expand_dims(img_array, axis=0)



# Make predictions
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
print(round(max(predictions[0])*100,2))
CLASS_NAMES[predicted_class]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
79.06


'Potato___Late_blight'

ValueError: None values not supported.